# LAB01

In [ ]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [ ]:
def generate_sets(t_prob, problem_size, num_sets):
  return tuple(np.array([random() < t_prob for _ in range(problem_size)]) for _ in range(num_sets))

In [ ]:
PROBLEM_SIZE = 25
NUM_SETS = 20
T_PROB = 0.15
SETS = generate_sets(T_PROB, PROBLEM_SIZE, NUM_SETS)

State = namedtuple('State', ['taken', 'not_taken'])

In [ ]:
def covered(state):
  return reduce(
    np.logical_or,
    [SETS[i] for i in state.taken],
    np.array([False for _ in range(PROBLEM_SIZE)]),
  )

def goal_check(state):
  return np.all(covered(state))

def print_result(result):
  print(f"{result[0]} solved in {result[1]:,} steps ({len(result[2])} tiles)")

# HEURISTICS

In [ ]:
def h01(state):
  already_covered = covered(state)
  if np.all(already_covered):
    return 0
  missing_size = PROBLEM_SIZE - sum(already_covered)
  candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
  taken = 1
  while sum(candidates[:taken]) < missing_size:
    taken += 1
  return taken

I tried to take a step forward with the optimistic distance heuristic above.\
With this heuristic (below) I tried to find a way to make the resulting cost more pessimistic, by calculating more precisely how many sets do we have to take to bring our agent to the goal.\
Sadly, this heuristic is suboptimal due to its non admissibility, because it can overestimate the cost from a state to the goal: this is caused by the fact that it takes the best matching as first set: it could possibly be that a worst matching set can lead to a minor cost.

In [ ]:
# try to take the information of the candidates down with the function call
def h02(state, j=0):
  already_covered = covered(state)
  if np.all(already_covered):
    return 0
  candidates = []
  for i in range(len(SETS)):
    candidates.append((sum(np.logical_and(SETS[i], np.logical_not(already_covered))), i))
  # take the best fitting set from the untaken
  candidate = max(candidates)[1]
  tmp_state = State(
      state.taken ^ {candidate},
      state.not_taken ^ {candidate},
    )
  if j>0:
    j-=1
  if j==0:
    taken = 1 + h01(tmp_state)
  else:
    taken = 1 + h02(tmp_state, j)
  return taken

# SOLVING

In [ ]:
def greedy(state):
  return PROBLEM_SIZE - sum(covered(state))
greedy_wrapper = "greedy", greedy

In [ ]:
def f_adm(state):
  return len(state.taken) + h01(state)
f_adm_wrapper = "f_adm", f_adm

In [ ]:
def f_not_adm(state):
  return len(state.taken) + h02(state)
f_not_adm_wrapper = "f_not_adm", f_not_adm

In [ ]:
def solve(wrapper):
  # unpacking solver characteristics 
  name = wrapper[0]
  f = wrapper[1]

  frontier = PriorityQueue()
  state = State(set(), set(range(NUM_SETS)))
  frontier.put((f(state), state))

  counter = 0
  _, current_state = frontier.get()
  with tqdm(total=None) as pbar:
    while not goal_check(current_state):
      counter += 1
      for action in current_state[1]:
        new_state = State(
          current_state.taken ^ {action},
          current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
      _, current_state = frontier.get()
      pbar.update(1)

  return name, counter, current_state.taken

In [ ]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"
iter = 10

for i in range(iter):
  # greedy
  print_result(solve(greedy_wrapper))

  # admissible heuristic A* 
  print_result(solve(f_adm_wrapper))

  # non admissible heuristic A* 
  print_result(solve(f_not_adm_wrapper))

  if(i < iter-1):
    SETS = generate_sets(T_PROB, PROBLEM_SIZE, NUM_SETS)
    assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"


Besides the suboptimality of the heuristic, it seems to perform pretty well in most of the cases

In [ ]:
# overlapping check utility function (not used)
def sum_sol(state):
  return np.array([SETS[i] for i in state.taken]).sum(0)